In [19]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)


llm = ChatOpenAI(model="gpt-4o-mini")

llm.with_structured_output()
def agent1(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

def agent2(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("agent1", agent1)
graph_builder.add_edge(START, "agent1")
graph_builder.add_edge("agent1", END)
graph = graph_builder.compile()

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [12]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


In [17]:
user_input = "cool"
stream_graph_updates(user_input)

Assistant: Glad you think so! What’s on your mind?
